# Predict Num_Mosquitos

#### Key variable missing from test data:  the number of mosquitos. To predict the number of mosquitos for this test data, we decided to create a model to predict the number of mosquitos using our train data.
#### First, define new y, NumMosquitos. Then, use rest of variables to predict for first pass. Expert knowledge might come in handy to reduce features if model is overfit.

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
%matplotlib inline

from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.optimizers import Adam

/Users/yukiharuhadeishi/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


#### Import Scaled Split Data

In [10]:
with open('../Cleansed_Data/X_train_ss.pkl','rb') as f:
    X_train_ss = pickle.load(f)
    
with open('../Cleansed_Data/X_test_ss.pkl','rb') as f:
    X_test_ss = pickle.load(f)
    
with open('../Cleansed_Data/y_train.pkl','rb') as f:
    y_train = pickle.load(f)
    
with open('../Cleansed_Data/y_test.pkl','rb') as f:
    y_test = pickle.load(f)

In [11]:
X_train_ss.head()

,Latitude,Longitude,NumMosquitos,Tavg,Tmax,Tmin,BR,HZ,RA,TSRA,...,Month_08,Month_09,Month_10,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
10140,-0.863790,-0.329696,0.011836,-0.002088,0.435022,-0.552766,-0.712498,-0.527474,-0.747020,-0.506378,...,-0.744753,1.931137,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
7737,0.515998,-0.176909,-0.546211,0.266496,0.495685,0.048510,-0.712498,-0.527474,-0.747020,-0.506378,...,-0.744753,1.931137,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
8079,-0.661906,1.428458,-0.732227,-2.822215,-2.962122,-2.356596,-0.712498,-0.527474,1.338653,-0.506378,...,-0.744753,1.931137,-0.162622,-0.011267,-0.593230,1.098637,-0.589125,-0.088332,-0.025199,-0.144431
2449,-1.013760,1.193503,2.306030,0.937954,0.980991,0.783404,1.403512,-0.527474,1.338653,1.974810,...,1.342727,-0.517830,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
449,0.188928,0.531668,-0.732227,0.535079,0.374358,0.716595,1.403512,1.895829,1.338653,1.974810,...,-0.744753,-0.517830,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431


In [12]:
# Define new y:  NumMosquitos
new_y_train_ss = X_train_ss.NumMosquitos

In [13]:
new_X_train_ss = X_train_ss.drop(columns='NumMosquitos')

In [14]:
new_X_train_ss.head()

,Latitude,Longitude,Tavg,Tmax,Tmin,BR,HZ,RA,TSRA,VCTS,...,Month_08,Month_09,Month_10,Species_CULEX ERRATICUS,Species_CULEX PIPIENS,Species_CULEX PIPIENS/RESTUANS,Species_CULEX RESTUANS,Species_CULEX SALINARIUS,Species_CULEX TARSALIS,Species_CULEX TERRITANS
10140,-0.863790,-0.329696,-0.002088,0.435022,-0.552766,-0.712498,-0.527474,-0.747020,-0.506378,-0.230349,...,-0.744753,1.931137,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
7737,0.515998,-0.176909,0.266496,0.495685,0.048510,-0.712498,-0.527474,-0.747020,-0.506378,-0.230349,...,-0.744753,1.931137,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
8079,-0.661906,1.428458,-2.822215,-2.962122,-2.356596,-0.712498,-0.527474,1.338653,-0.506378,-0.230349,...,-0.744753,1.931137,-0.162622,-0.011267,-0.593230,1.098637,-0.589125,-0.088332,-0.025199,-0.144431
2449,-1.013760,1.193503,0.937954,0.980991,0.783404,1.403512,-0.527474,1.338653,1.974810,-0.230349,...,1.342727,-0.517830,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431
449,0.188928,0.531668,0.535079,0.374358,0.716595,1.403512,1.895829,1.338653,1.974810,4.341238,...,-0.744753,-0.517830,-0.162622,-0.011267,1.685687,-0.910219,-0.589125,-0.088332,-0.025199,-0.144431


# Linear Regression to predict NumMosquitos: 

In [15]:
linreg = LinearRegression()

In [16]:
linreg.fit(new_X_train_ss, new_y_train_ss)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
NumMosq_hat = linreg.predict(new_X_train_ss)

In [18]:
type(NumMosq_hat)

numpy.ndarray

In [19]:
len(new_y_train_ss)

7879

In [20]:
NumMosq_train_df = pd.DataFrame(new_y_train_ss)

In [21]:
index = NumMosq_train_df.index

TypeError: Index(...) must be called with a collection of some kind, 'NumMosq_Predicted' was passed